In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch
from peft import PeftConfig, PeftModel
import gradio as gr

tokenizer = AutoTokenizer.from_pretrained("baskarm/deepseek-coder-7b-instruct-v1.5_finetune")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    )
config = PeftConfig.from_pretrained("baskarm/deepseek-coder-7b-instruct-v1.5_finetune")
base_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5", quantization_config=bnb_config).eval()
model = PeftModel.from_pretrained(base_model, "baskarm/deepseek-coder-7b-instruct-v1.5_finetune")

/home/baskar/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:03<00:00,  1.15s/it]


In [2]:
def code_gen(question, history):
    messages=[
        { 'role': 'user', 'content': question}
    ]
    
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [3]:
js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '120px';

    var text = 'Code-Gen+Completion';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.05s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);
    document.body.style.backgroundImage = "url(https://companiesmarketcap.com/img/company-logos/256/CALX.webp)";
    return 'Animation created';
}
"""
css=""".gradio-container{
  /* The image used */
  background-image: url(https://companiesmarketcap.com/img/company-logos/256/CALX.webp);
  /* Center and scale the image nicely */
  object-position : top-left;
  background-repeat: no-repeat;
}"""
demo = gr.ChatInterface(code_gen, css=css, js=js)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.27.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://a620c4e56cfb9230f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/baskar/miniconda3/envs/llm/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:538: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results